In [1]:
import numpy
import pandas as pd
import seaborn as sns
import random
import math
import os

from IPython import display

pd.set_option('display.max_columns', None)

### Import model objects from their unique directories - easier to organize

from commuter_model.commuter_model import CommuterModel
from electrical_model.electrical_model import ElectricModel

In [2]:
# load ipums data to local variable (ipums_df_19)
ipums_df_19 = pd.read_csv("commuter_model/disaggregated_cleaned_ipums_data.csv",index_col=0)
ipums_df_19 = ipums_df_19[ipums_df_19['YEAR']==2019].reset_index(drop=True).copy()



In [3]:
commuter_model_alltransit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_allmicro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_mix = CommuterModel(ipums_df=ipums_df_19.copy())

In [4]:
### Zeroing out non-transit
commuter_model_alltransit.ipums_df['FLAG_AUTO'] = 0
commuter_model_alltransit.ipums_df['FLAG_ESCOOTER'] = 0
commuter_model_alltransit.ipums_df['FLAG_WALK'] = 0
commuter_model_alltransit.ipums_df['FLAG_EBIKE']= 0
commuter_model_alltransit.ipums_df['FLAG_MOTORCYCLE']= 0
commuter_model_alltransit.ipums_df['FLAG_TAXICAB']= 0

# print(commuter_model_alltransit.ipums_df[['FLAG_AUTO',"FLAG_ESCOOTER","FLAG_WALK","FLAG_EBIKE"]].sum())

#### Transit
### Bus
commuter_model_alltransit.ipums_df['FLAG_EBUSES'] = commuter_model_alltransit.bus_flag_binary(
                                        home_region=commuter_model_alltransit.puma_home_Bus
                                        ,work_region=commuter_model_alltransit.puma_work_Bus
                                        ,schedule=commuter_model_alltransit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_alltransit.ipums_df['FLAG_SUBWAY'] = commuter_model_alltransit.subway_flag_binary(
                                            home_region=commuter_model_alltransit.puma_home_Subway
                                            ,work_region=commuter_model_alltransit.puma_work_Subway
                                            ,schedule=commuter_model_alltransit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_alltransit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_alltransit.commuterRail_flag_binary(
                                             home_region=commuter_model_alltransit.puma_home_CommuterRail
                                            ,work_region=commuter_model_alltransit.puma_work_CommuterRail
                                            ,schedule=commuter_model_alltransit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_alltransit.ipums_df['FLAG_FERRY'] = commuter_model_alltransit.ferry_flag_binary(
                                            home_region=commuter_model_alltransit.puma_home_Ferry
                                           ,work_region=commuter_model_alltransit.puma_work_Ferry
                                           ,schedule=commuter_model_alltransit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Work From Home
commuter_model_alltransit.ipums_df["FLAG_WFH"] = commuter_model_alltransit.wfh_flag_binary(
                                            wfh_dampener=1
                                                )

# print(commuter_model_alltransit.ipums_df[flag_cols].sum())


commuter_model_alltransit.ipums_df['First_Assignment'] = commuter_model_alltransit.RandAssignNoOption(
                                                                commuter_model_alltransit.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_alltransit.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_alltransit.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_alltransit.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_alltransit.ipums_df['FLAG_WALK'], 
                                                                commuter_model_alltransit.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_alltransit.ipums_df['FLAG_WFH'])
commuter_model_alltransit.ipums_df['TransMode'] = commuter_model_alltransit.RandAssignChosenOption(
                                                                commuter_model_alltransit.ipums_df['First_Assignment'], 
                                                                commuter_model_alltransit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [5]:
flag_cols = [x for x in commuter_model_alltransit.ipums_df if "FLAG" in x]


In [6]:
commuter_model_alltransit.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              6539
dtype: int64

In [7]:
commuter_model_alltransit.ipums_df["First_Assignment"].value_counts()

Subway          9090
Bus             6101
CommuterRail    3871
WFH             2761
No option       1253
Ferry            902
Name: First_Assignment, dtype: int64

In [8]:
commuter_model_alltransit.ipums_df["TransMode"].value_counts()

Subway           9280
Bus              6335
CommuterRail     4139
WFH              2766
Ferry             936
AutoOccupants     469
Walk               31
Other              15
Bicycle             7
Name: TransMode, dtype: int64

In [9]:
commuter_model_alltransit.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              6539
dtype: int64

In [10]:
#### All Micro-Mobility
commuter_model_allmicro.ipums_df['FLAG_AUTO'] = 0
## Motorcycle
commuter_model_allmicro.ipums_df['FLAG_MOTORCYCLE'] = 0
## Taxicab
commuter_model_allmicro.ipums_df['FLAG_TAXICAB'] = 0
## Bus
commuter_model_allmicro.ipums_df['FLAG_EBUSES'] = 0
## Subway
commuter_model_allmicro.ipums_df['FLAG_SUBWAY'] = 0
## Commuter Rail
commuter_model_allmicro.ipums_df['FLAG_COMMUTERRAIL'] = 0
## Ferry
commuter_model_allmicro.ipums_df['FLAG_FERRY'] = 0


## E-Scooter - bump up distance
commuter_model_allmicro.ipums_df['FLAG_ESCOOTER'] = commuter_model_allmicro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_allmicro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_allmicro.ipums_df['FLAG_WALK'] = commuter_model_allmicro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_allmicro.ipums_df['FLAG_EBIKE']=commuter_model_allmicro.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_allmicro.bike_friendly_origins
                                        )


## Work From Home
commuter_model_allmicro.ipums_df["FLAG_WFH"] = commuter_model_allmicro.wfh_flag_binary(
                                            wfh_dampener=1
                                                )

commuter_model_allmicro.ipums_df['First_Assignment'] = commuter_model_allmicro.RandAssignNoOption(
                                                                commuter_model_allmicro.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_allmicro.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_allmicro.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_allmicro.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_allmicro.ipums_df['FLAG_WALK'], 
                                                                commuter_model_allmicro.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_allmicro.ipums_df['FLAG_WFH'])
commuter_model_allmicro.ipums_df['TransMode'] = commuter_model_allmicro.RandAssignChosenOption(
                                                                commuter_model_allmicro.ipums_df['First_Assignment'], 
                                                                commuter_model_allmicro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [11]:
commuter_model_allmicro.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14140
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES              0
FLAG_SUBWAY              0
FLAG_COMMUTERRAIL        0
FLAG_FERRY               0
FLAG_WFH              6501
dtype: int64

In [12]:
commuter_model_allmicro.ipums_df["First_Assignment"].value_counts(dropna=False)

Bicycle      8498
No option    6872
WFH          4337
Escooter     3860
Walk          411
Name: First_Assignment, dtype: int64

In [13]:
commuter_model_allmicro.ipums_df["TransMode"].value_counts()

Bicycle          8518
WFH              4361
Escooter         3860
AutoOccupants    1789
Subway           1778
CommuterRail     1698
Bus              1226
Walk              519
Ferry             199
Other              30
Name: TransMode, dtype: int64

In [14]:
#### Standard

commuter_model_mix.ipums_df['FLAG_AUTO'] = commuter_model_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )


## Taxicab
commuter_model_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )

## Bus
commuter_model_mix.ipums_df['FLAG_EBUSES'] = commuter_model_mix.bus_flag_binary(
                                        home_region=commuter_model_mix.puma_home_Bus
                                        ,work_region=commuter_model_mix.puma_work_Bus
                                        ,schedule=commuter_model_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_mix.subway_flag_binary(
                                            home_region=commuter_model_mix.puma_home_Subway
                                            ,work_region=commuter_model_mix.puma_work_Subway
                                            ,schedule=commuter_model_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_mix.ipums_df['FLAG_FERRY'] = commuter_model_mix.ferry_flag_binary(
                                            home_region=commuter_model_mix.puma_home_Ferry
                                           ,work_region=commuter_model_mix.puma_work_Ferry
                                           ,schedule=commuter_model_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_mix.bike_friendly_origins
                                            ,male_pct=25
                                            ,female_pct=25
                                            )


## Walking
commuter_model_mix.ipums_df['FLAG_WALK'] = commuter_model_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_mix.ipums_df['FLAG_EBIKE']=commuter_model_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_mix.bike_friendly_origins
                                        )


## Work From Home
commuter_model_mix.ipums_df["FLAG_WFH"] = commuter_model_mix.wfh_flag_binary(
                                            wfh_dampener=.5
                                                )


commuter_model_mix.ipums_df['First_Assignment'] = commuter_model_mix.RandAssignNoOption(
                                                                commuter_model_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_mix.ipums_df['FLAG_WFH'])
commuter_model_mix.ipums_df['TransMode'] = commuter_model_mix.RandAssignChosenOption(
                                                                commuter_model_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [15]:
commuter_model_mix.ipums_df[flag_cols].sum()

FLAG_AUTO             1688
FLAG_ESCOOTER          244
FLAG_WALK             1082
FLAG_EBIKE            7188
FLAG_MOTORCYCLE         34
FLAG_TAXICAB           315
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              3322
dtype: int64

In [16]:
commuter_model_mix.ipums_df["First_Assignment"].value_counts()

Subway           7666
Bus              5406
CommuterRail     3870
Bicycle          2490
WFH              1253
No option        1251
Ferry             921
AutoOccupants     663
Walk              295
Taxicab            86
Escooter           67
Motorcycle         10
Name: First_Assignment, dtype: int64

In [17]:
commuter_model_mix.ipums_df["TransMode"].value_counts()

Subway           7837
Bus              5629
CommuterRail     4210
Bicycle          2496
WFH              1256
AutoOccupants    1093
Ferry             950
Walk              327
Taxicab            86
Escooter           67
Other              17
Motorcycle         10
Name: TransMode, dtype: int64

In [18]:
#### Compare Counts of Flags
# commuter_model_alltransit.ipums_df['FLAG_AUTO'].sum()
ct_1 = commuter_model_allmicro.ipums_df['First_Assignment'].value_counts()

# commuter_model_alltransit.ipums_df['FLAG_AUTO'].sum()
ct_2 = commuter_model_alltransit.ipums_df['First_Assignment'].value_counts()

# commuter_model_mix.ipums_df['FLAG_AUTO'].sum()
ct_3 =commuter_model_mix.ipums_df['First_Assignment'].value_counts()

pd.merge(left=ct_1,right=ct_2,left_index=True,right_index=True,how='outer')\
.merge(right=ct_3,left_index=True,right_index=True,how='outer')

,First_Assignment_x,First_Assignment_y,First_Assignment
AutoOccupants,NaN,NaN,663
Bicycle,8498.0,NaN,2490
Bus,NaN,6101.0,5406
CommuterRail,NaN,3871.0,3870
Escooter,3860.0,NaN,67
Ferry,NaN,902.0,921
Motorcycle,NaN,NaN,10
No option,6872.0,1253.0,1251
Subway,NaN,9090.0,7666
Taxicab,NaN,NaN,86


In [19]:
commuter_model_alltransit.ipums_df.groupby(by=["YEAR"])[flag_cols].sum()
commuter_model_alltransit.ipums_df['First_Assignment'].value_counts()

Subway          9090
Bus             6101
CommuterRail    3871
WFH             2761
No option       1253
Ferry            902
Name: First_Assignment, dtype: int64

In [21]:
master_df = pd.DataFrame()
for x in [commuter_model_alltransit,commuter_model_allmicro,commuter_model_mix]:
    master_df = pd.concat([master_df,x.ipums_df.groupby(by=["TransMode"])['PERWT'].sum()],axis=1)
master_df.columns=["all_transit_count","all_micro_count","mix_count"]
master_df

,all_transit_count,all_micro_count,mix_count
AutoOccupants,48155.0,187782.0,108085.0
Bicycle,540.0,1060871.0,312136.0
Bus,779793.0,134606.0,704357.0
CommuterRail,423551.0,169623.0,429519.0
Ferry,94750.0,19462.0,96118.0
Other,1831.0,3063.0,1959.0
Subway,1148144.0,169788.0,933858.0
WFH,301444.0,445954.0,138217.0
Walk,3122.0,85510.0,52078.0
Escooter,NaN,524671.0,10684.0


In [22]:
### Electric Model
### read in commuter_df. feed that and the ev file location to class
em_1 = ElectricModel(commuter_model_alltransit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3,bus_ferry_cab_delay=6)
em_2 = ElectricModel(commuter_model_allmicro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3,bus_ferry_cab_delay=6)
em_3 = ElectricModel(commuter_model_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3,bus_ferry_cab_delay=6)

In [23]:
ct= 0 
for e_m in [em_1,em_2,em_3]:
    ct+=1
    ## Run methods in order to read in, clean, and aggregate data
    e_m.read_electric_info_in()

    e_m.clean_commuter_df()

    e_m.create_in_out_flow_dfs()

    e_m.aggregate_data()
    print(f"electric model done for em {ct}")

electric model done for em 1
electric model done for em 2
electric model done for em 3


In [24]:
save_dir = "../06_Dashboard_Visualization/electric_model_outputs"
os.makedirs(save_dir,exist_ok=True)
em_1.df_aggregate.to_pickle(f"{save_dir}/electric_model_alltransit_dfaggregate.pkl")
em_2.df_aggregate.to_pickle(f"{save_dir}/electric_model_allmicro_dfaggregate.pkl")
em_3.df_aggregate.to_pickle(f"{save_dir}/electric_model_mix_dfaggregate.pkl")

In [25]:
sorted(em_1.df_aggregate.PEV_DELAY.dropna().unique())

['Custom', 'Earliest', 'Latest', 'Random']

In [26]:
sorted(em_1.df_aggregate.TransMode.dropna().unique())

['AutoOccupants', 'Bicycle', 'Bus', 'CommuterRail', 'Ferry', 'Subway']

In [ ]:
# # test_df = em_1.df_aggregate.copy()
# test_df[(test_df['Charge_Hour']==0)\
#        ]
# #        &(test_df['TransMode']=="Bus")]